## CPSC 477 Final Project Part 3: Evaluation

In [14]:
import os
import evaluate
from util import *
from openai import OpenAI
from dotenv import load_dotenv
env = load_dotenv(override = True)
rouge = evaluate.load("rouge")

In [26]:
# Obtained base model inference on test dataset 
# Took 20 min on V100 (Google Colab)
model = "base_mistral"

test_filenames = os.listdir(f"inference/{model}")
if ".ipynb_checkpoints" in test_filenames:
    test_filenames.remove(".ipynb_checkpoints")

candidates = []
references = []
ects = []
for filename in test_filenames:
    base_inference = ""
    gemini_summary = ""
    ect = ""
    with open(f"inference/{model}/{filename}") as f:
        base_inference = f.read()
    with open(f"dataset/test/gemini_summaries/{filename}") as f:
        gemini_summary = f.read()
    with open(f"dataset/test/ects/{filename}") as f:
        ect = f.read()
    candidates.append(base_inference)
    references.append(gemini_summary)
    ects.append(ect)

print(len(candidates), len(references))

50 50


In [27]:
results = rouge.compute(predictions=candidates, references=references)
print(results)

{'rouge1': 0.4499200911067884, 'rouge2': 0.18716517254656978, 'rougeL': 0.26825319469884557, 'rougeLsum': 0.2729071929061325}


In [13]:
# Summary numbers - precision and recall
for i, filename in enumerate(test_filenames):
    ect_numbers = extract_numbers(ects[i])
    candidate_numbers = extract_numbers(candidates[i])
    reference_numbers = extract_numbers(references[i])
    # Recall
    \

# Transcript numbers - precision and recall

c [(66.0, '%'), (3.71, '$'), (50.0, '%'), (88.0, '$'), (68.0, '%'), (36.0, '%'), (64.0, '%'), (11.7, '%'), (220.0, None), (217.0, '$'), (1.85, '$'), (500.0, '$')]
Watsco has delivered an incredible second quarter, achieving new records in virtually every performance metric. Earnings per share jumped 64% to a record $3.71 per share on a 66% increase in net income. Sales grew 36% or nearly $500 million to a record $1.85 billion in sales for the quarter. Gross profits increased 50% with gross margins expanding 220 basis points. Operating income increased $88 million or 68% to $217 million. Operating margins expanded 220 basis points to a record 11.7%. These results are all the more positive when considered against last year's second quarter, which had only a modest impact from the COVID-related slowdowns. Watsco's industry-leading technologies continue to gain traction, and the company believes they are helping them gain market share. Growth rates among active users of their technologies 

In [ ]:
# Generate summaries with GPT-3.5
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

prompt = "Take this scraped input for a clothing item and list the following in the following format. For fields that you do not find, write N/A.\nFormat:\nMaterial: [X]% [Cotton or Organic Cotton or Polyester or Lyocell or Elastane or Polyamide or Elastomultiester], [X]% [Cotton or Organic Cotton or Polyester or Lyocell or Elastane or Polyamide or Elastomultiester or Other]…\nRecycled Material: [X]%\nCountry of Origin: [United States or Laos or Vietnam…]\nCompany: [SHEIN or Amazon or GAP…]\nClothing Item: [string]"

response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript."},
        {"role": "user", "Below is an earnings call transcript. Please summarize this transcript in exactly one paragraph using complete sentences. Keep the summary below 300 words. It is very important that you do not use any titles in the summary. Include relevant information and statistics from the Earnings Call Transcript in your summary.\n\nEarnings Call Transcript:\n{}"},
    ]
)
print(response)